In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import random

In [2]:
batch_size=256
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128
result_path="results_np_rz_mu/"

In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [5]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
        
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.latent_layer_1=nn.Linear(3,400)
        self.latent_layer_2=nn.Linear(400,400)
        self.latent_layer_3=nn.Linear(400,self.r_dim)
        
        self.s_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.s_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.r_dim+ self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400,400)
        self.g_3 = nn.Linear(400,400)
        self.g_mu = nn.Linear(400, 1)
        self.g_logvar = nn.Linear(400, 1)

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def latent(self, x_y):
        x_y = F.relu(self.latent_layer_1(x_y))
        x_y = F.relu(self.latent_layer_2(x_y))
        x_y = F.relu(self.latent_layer_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def reparameterise(self, z):
        mu, std= z
        eps = torch.randn_like(std)
        z_sample = eps.mul(std).add_(mu)
        z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
        return z_sample

    def g(self, r,z_sample, x_target):
        rz_et_x = torch.cat([r,z_sample, x_target], dim=2)
        input = F.relu(self.g_1(rz_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        y_mu=self.g_mu(input)
        y_logvar=self.g_logvar(input)
        y_var=0.1+0.9*F.softplus(y_logvar)
        return y_mu,y_var
    
    def xy_to_r_params(self, x, y):       
        x_y = torch.cat([x, y], dim=2)
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        return r
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        s_i = self.latent(x_y)
        s = self.aggregate(s_i)
        z_mu = self.s_to_z_mean(s)
        z_logvar = self.s_to_z_logvar(s)
        z_sigma=0.1+0.9*F.softplus(z_logvar)
        return z_mu, z_sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        #produire r
        r_context = self.xy_to_r_params(x_context, y_context)
        r_expand = r_context.unsqueeze(1).expand(-1, 784, -1)
        #produire z
        z_context = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_all = self.xy_to_z_params(x_all, y_all)  
        else:  # at test time we don't have the image so we use only the context
            z_all = z_context
        z_sample = self.reparameterise(z_all)
        
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        y_mu, y_sigma= self.g(r_expand,z_sample, x_target)
    
        
        return y_mu,y_sigma,z_all, z_context

In [6]:
def kl_div_gaussians(mu_q, logvar_q, mu_p, logvar_p):
    var_p = torch.exp(logvar_p)
    kl_div = (torch.exp(logvar_q) + (mu_q - mu_p) ** 2) / var_p \
             - 1.0 \
             + logvar_p - logvar_q
    kl_div = 0.5 * kl_div.sum()
    return kl_div


def np_loss(mu,sigma, y, z_all, z_context):
    # get log probability
    logprob=-(y - mu)**2 / (2 * sigma**2) - torch.log(sigma)
    
    # get KL divergence between prior and posterior
    KLD = kl_div_gaussians(z_all[0], z_all[1], z_context[0], z_context[1])
    # maximize prob and minimize KL divergence
    return -logprob + KLD


In [7]:
model = NP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        mu,sigma, z_all, z_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(mu,sigma, y_all, z_all, z_context).sum(dim=0).mean()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [9]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            mu,sigma, z_all, z_context = model(x_context, y_context)
            test_loss += np_loss(mu,sigma, y_all, z_all, z_context).sum(dim=0).mean().item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    recons_y=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):
                        mu,sigma, _, _ = model(x_context, y_context)
                        y_hat=torch.normal(mu,sigma)
                        recons.append(mu[:num_examples])
                        recons1.append(sigma[:num_examples])
                        recons_y.append(y_hat[:num_examples])
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons_y = torch.cat(recons_y).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1,recons_y])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [10]:
for epoch in range(3, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 3 [0/60000 (0%)]	Loss: -0.159592


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), result_path+"model.pt")